<h1 style="color:blue">Praktikum 3. Morfoloogilise analüüsi erijuhud</h1>
<h3 style="color:blue">Korpusepõhine ühestamine, morfoloogiline analüüs kasutajasõnastiku abil ja Giellatekno märgendid</h3>

Tänases praktikumis jätkame morfoloogilise analüüsi ja ühestamise teemadega. Kuna keerukamad analüüsitasemed (süntaks, semantika) toetuvad morfoloogilise analüüsi tulemustele, siis on oluline selle taseme analüüsile rohkem tähelepanu pöörata. Eriti oluliseks muutub see siis, kui analüüsida tuleb tekste, mille keelekasutus erineb kirjakeelest, nt netikeelt, slängi vms eesti keele allkeelt. Samuti võib sõnade morfoloogilist analüüsi parandada see, kui arvestame sõnade kasutust laiemas kontekstis: terves tekstis või _korpuses_ ¹.

   * ¹ _korpus_ = loomuliku keele tekstide kogu. Täpsema määratluse korpuste kohta lingvistikas leiab [siit](https://et.wikipedia.org/wiki/Keelekorpus).

## Morfoloogiline ühestamine

Vaikimisi EstNLTK's kasutatav morfoloogiline ühestamine, millega tutvusite eelmises praktikumis, toimetab ühe lause piires **(seetõttu on analüüsiks vaja ka kihti *sentences*)**.  Lühidalt kirjeldades: statistilist / masinõppe lähenemist kasutades leitakse igale sõnale kontekstist lähtuvalt kõige tõenäolisem morfoloogiline analüüs, sealjuures üksikule sõnale analüüsi valimisel lähtutakse 1-2 eelneva sõna analüüsidest. Detailsemalt kirjeldab statistilist morfoloogilist ühestamist [see artikkel](http://www.cl.ut.ee/yllitised/kk_yhest_1998.pdf).

Praktikas töötab lausepõhine ühestamine küllaltki hästi, aga selle täpsust on võimalik siiski veelgi suurendada. Ühe lause piires mitmeseks jäävad sõnad võivad mujal tekstis esineda üheselt tõlgendataval kujul (nt tähenduslikud (perekonna)nimed), seega, kui kasutada ühestamisel laiemat konteksti kui üks lause, saame lahendada lausepõhise ühestamise poolt lahendamata jäänud või valesti lahendatud mitmesusi.

Uurime lause- ja korpusepõhise morfoloogilise ühestamise erinevust järgmise "näitekorpuse" varal:

In [1]:
corpus_texts = ['Esimesele kohale tuleb Jänes, kuigi tema punktide summa pole kõrgeim.',\
          'Lõpparvestuses läks Konnale esimene koht. Teise koha sai seekord Jänes. Uus võistlus toimub 2. mail.', \
          'Konn paistis silma suurima punktide summaga. Uue võistluse toimumisajaks on 2. mai.']

Järjendi `corpus_texts` iga elementi tuleks käsitleda "ühe tekstina". Selleks muudame laused korpuses `Text`-objektideks ja rakendame meetodi `tag_layer()` kaudu morfoloogilise analüüsi tavalise, lausepõhise ühestamisega.  Sisuliselt teostab see meetod morfoloogilise analüüsi vastavalt sellele, milline `resolver` on kasutusel teksti märgendamisel *tag_layer* meetodiga ehk millised olid *resolver*'i parameetrite `guess`, `disambiguate` ja `propername` väärtused (eelmise praktikumi teema). Kui *resolver* ühtegi nimetatud parameetritest ei määra on vaikimisi kõik sisse lülitatud. 

Kasutame kõige selle tegemiseks väikest [tsükli-nõksu](https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions).

In [2]:
from estnltk import Text
corpus = [Text(t).tag_layer(['morph_analysis']) for t in corpus_texts]
corpus

[Text(text='Esimesele kohale tuleb Jänes, kuigi tema punktide summa pole kõrgeim.'),
 Text(text='Lõpparvestuses läks Konnale esimene koht. Teise koha sai seekord Jänes. Uus võistlus toimub 2. mail.'),
 Text(text='Konn paistis silma suurima punktide summaga. Uue võistluse toimumisajaks on 2. mai.')]

Pärast teksti ühestamist trükime välja sõnad, mille analüüsid jäid mitmeseks. Kuvame eelnevalt tutvustatud nõksu abil morfoloogilise analüüsi tulemused nii, et oluline info oleks ühel real. Väljastame iga sõna analüüside osast ainult lemma, sõnaliigi ja käände- või pöördeinfo:

In [3]:
for text in corpus:
    print(text.text)
    # Trükime välja mitmeseks jäänud sõnade analüüsid (ainult lemma, sõnaliigi ja vorminimetuse)
    for word in text.morph_analysis:
        if len(word.annotations) > 1:
            print(word.text, '=>',[(a.lemma, a.partofspeech, a.form) for a in word.annotations] )
    print()

Esimesele kohale tuleb Jänes, kuigi tema punktide summa pole kõrgeim.
kohale => [('koha', 'S', 'sg all'), ('koht', 'S', 'sg all')]
kuigi => [('kuigi', 'D', ''), ('kuigi', 'J', '')]

Lõpparvestuses läks Konnale esimene koht. Teise koha sai seekord Jänes. Uus võistlus toimub 2. mail.
Teise => [('teine', 'O', 'sg g'), ('teine', 'P', 'sg g')]
koha => [('koha', 'S', 'sg g'), ('koht', 'S', 'sg g')]
mail => [('maa', 'S', 'pl ad'), ('mai', 'S', 'sg ad')]

Konn paistis silma suurima punktide summaga. Uue võistluse toimumisajaks on 2. mai.
summaga => [('summ', 'S', 'sg kom'), ('summa', 'S', 'sg kom')]
on => [('olema', 'V', 'b'), ('olema', 'V', 'vad')]



### Korpusepõhine ühestamine


Nagu näha, on ühestamise tulemustes parandamisruumi küll. Proovime korpusepõhist ühestamist. Selleks käsitleme korpust Text-objektide kogumina ja impordime EstNLTK-st klassi VabamorfCorpusTagger:

In [4]:
from estnltk.taggers import VabamorfCorpusTagger

Loome tagger'i ning määrame selle lisatava kihi:

In [5]:
vm_corpus_tagger = VabamorfCorpusTagger(output_layer='morph_analysis_with_cb_disamb')

Märgendame teksti uuesti korpusõhise ühestajaga

In [6]:
vm_corpus_tagger.tag(corpus)

[Text(text='Esimesele kohale tuleb Jänes, kuigi tema punktide summa pole kõrgeim.'),
 Text(text='Lõpparvestuses läks Konnale esimene koht. Teise koha sai seekord Jänes. Uus võistlus toimub 2. mail.'),
 Text(text='Konn paistis silma suurima punktide summaga. Uue võistluse toimumisajaks on 2. mai.')]

Uurime tulemusi – väljastame uuesti mitmesed analüüsid:

In [7]:
for text in corpus:
    print(text.text)
    # Trükime välja mitmeseks jäänud sõnade analüüsid (ainult lemma, sõnaliigi ja vorminimetuse)
    for word in text.morph_analysis_with_cb_disamb:
        if len(word.annotations) > 1:
            print( word.text,'=>',[(a.lemma, a.partofspeech, a.form) for a in word.annotations] )
    print()

Esimesele kohale tuleb Jänes, kuigi tema punktide summa pole kõrgeim.
kuigi => [('kuigi', 'D', ''), ('kuigi', 'J', '')]

Lõpparvestuses läks Konnale esimene koht. Teise koha sai seekord Jänes. Uus võistlus toimub 2. mail.
Teise => [('teine', 'O', 'sg g'), ('teine', 'P', 'sg g')]

Konn paistis silma suurima punktide summaga. Uue võistluse toimumisajaks on 2. mai.
on => [('olema', 'V', 'b'), ('olema', 'V', 'vad')]



Lisaks on korpusepõhisest ühestamisest abi pärisnimeanalüüside korrastamisel. Kui mingi tavaline sõna on korpuse tekstides tõenäoliselt kasutusel pärisnimena (st esineb suure algustähega ka lausete keskel), siis valib korpusepõhine ühestaja pärisnime-analüüsid selle sõna tavapäraste analüüside asemel. Uurime, millised analüüsid valiti näitekorpuse suure tähega algavatele sõnadele nii vaikimisi kui ka korpuspõhisel ühestamisel:

In [8]:
for text in corpus:
    print(text.text)
    # Trükime välja suure tähega algavate sõnade analüüsid
    for word in text.words:
        if word.text.istitle():
            print( "Vaikimisi:", word.text,'=>',[(a.lemma, a.partofspeech, a.form) for a in word.morph_analysis.annotations] )
            print( "Korpusepõhine:", word.text,'=>',[(a.lemma, a.partofspeech, a.form) 
                                   for a in word.morph_analysis_with_cb_disamb.annotations] )
    print()

Esimesele kohale tuleb Jänes, kuigi tema punktide summa pole kõrgeim.
Vaikimisi: Esimesele => [('esimene', 'O', 'sg all')]
Korpusepõhine: Esimesele => [('esimene', 'O', 'sg all')]
Vaikimisi: Jänes => [('jänes', 'S', 'sg n')]
Korpusepõhine: Jänes => [('Jänes', 'H', 'sg n')]

Lõpparvestuses läks Konnale esimene koht. Teise koha sai seekord Jänes. Uus võistlus toimub 2. mail.
Vaikimisi: Lõpparvestuses => [('lõpparvestus', 'S', 'sg in')]
Korpusepõhine: Lõpparvestuses => [('lõpparvestus', 'S', 'sg in')]
Vaikimisi: Konnale => [('konn', 'S', 'sg all')]
Korpusepõhine: Konnale => [('Konn', 'H', 'sg all')]
Vaikimisi: Teise => [('teine', 'O', 'sg g'), ('teine', 'P', 'sg g')]
Korpusepõhine: Teise => [('teine', 'O', 'sg g'), ('teine', 'P', 'sg g')]
Vaikimisi: Jänes => [('jänes', 'S', 'sg n')]
Korpusepõhine: Jänes => [('Jäne', 'H', 'sg in')]
Vaikimisi: Uus => [('uus', 'A', 'sg n')]
Korpusepõhine: Uus => [('uus', 'A', 'sg n')]

Konn paistis silma suurima punktide summaga. Uue võistluse toimumisajaks 

Nagu näha, on sõnad _Jänes_ ja _Konn_ saanud korpuspõhisel ühestamisel pärisnime sõnaliigimärgendi (`'H'`).

### Tekstipõhine ühestamine

Vahel võivad korpuse tekstid olla väga heterogeensed (näiteks erinevatel teemadel artiklid ühes ajakirjanduskorpuses) ja tekib oht, et sama sõna esineb kahes tekstis täiesti erinevates tähendustes. Sel juhul võib aga juhtuda, et liiga lai ühestamiskontekst (kogu korpus) ajab lemmad valeks. 

Kui kogu korpuse põhjal ühestamine tundubki liiga lai, on võimalik ühestada tekstipõhiselt, kasutades korpusepõhist ühestajat. See tähendab, et käsitleme iga korpuse teksti kui eraldiseisvat korpust, andes ühestajale ette vaid ühest tekstist koosneva järjendi:

In [9]:
print(corpus)

[Text(text='Esimesele kohale tuleb Jänes, kuigi tema punktide summa pole kõrgeim.'), Text(text='Lõpparvestuses läks Konnale esimene koht. Teise koha sai seekord Jänes. Uus võistlus toimub 2. mail.'), Text(text='Konn paistis silma suurima punktide summaga. Uue võistluse toimumisajaks on 2. mai.')]


In [10]:
vm_tb_tagger = VabamorfCorpusTagger(output_layer='text_based_disamb')

for corpus_text in corpus:
    vm_tb_tagger.tag([corpus_text])  # käsitleme iga Text-objekti eraldi korpusena

Võrdleme korpuspõhise ja tekstipõhise ühestamise kaudu saadud märgenduste erinevusi:

In [11]:
# Korpuspõhise ja tekstipõhise ühestaja mittekattuvad märgendused
for corpus_text in corpus:
    for i, analysis in enumerate(corpus_text.morph_analysis_with_cb_disamb):
        if corpus_text.text_based_disamb[i] != analysis:
            display(analysis)
            display(corpus_text.text_based_disamb[i])
            print('*'*50)

text,normalized_text,lemma,root,root_tokens,ending,clitic,form,partofspeech
kohale,kohale,koht,koht,['koht'],le,,sg all,S


text,normalized_text,lemma,root,root_tokens,ending,clitic,form,partofspeech
kohale,kohale,koha,koha,['koha'],le,,sg all,S
,kohale,koht,koht,['koht'],le,,sg all,S


**************************************************


text,normalized_text,lemma,root,root_tokens,ending,clitic,form,partofspeech
Konnale,Konnale,Konn,Konn,['Konn'],le,,sg all,H


text,normalized_text,lemma,root,root_tokens,ending,clitic,form,partofspeech
Konnale,Konnale,Konnale,Konnale,['Konnale'],0,,sg g,H


...

text,normalized_text,lemma,root,root_tokens,ending,clitic,form,partofspeech
Konn,Konn,konn,konn,['konn'],0,,sg n,S


**************************************************


text,normalized_text,lemma,root,root_tokens,ending,clitic,form,partofspeech
summaga,summaga,summa,summa,['summa'],ga,,sg kom,S


text,normalized_text,lemma,root,root_tokens,ending,clitic,form,partofspeech
summaga,summaga,summ,summ,['summ'],ga,,sg kom,S
,summaga,summa,summa,['summa'],ga,,sg kom,S


**************************************************


Kuna näites toodud tekstid olid omavahel tugevasti seotud, on näha, et parema tulemuse andis seekord korpusepõhine ühestamine ja tekstipõhiselt jäid nii mõnedki sõnad mitmeseks.

### Lisaks: kahetasemeline korpusepõhine ühestamine

Korpusepõhist ühendajat saab kasutada veel kolmandatki moodi: EstNLTK võimaldab eksperimenteerida kahetasemelise korpusepõhise ühestamisega. Seda tasub katsetada samuti pigem heterogeensetel tekstidel. Kui meil on mingi alus, mille abil tekste võiks väiksemateks ja seotumateks osadeks liigitada, nt ajalehekorpuses artikli kuupäev või rubriik, on kahetasemelisel korpusepõhisel ühestamisel võimalik esmalt ühestada tekstid väiksema üksuse ning seejärel kogu korpuse põhjal. Kuna tegu on eksperimentaalse võimalusega, pole veel teada, kui palju selline lähenemine tulemusi parandab.

Kahetasemeliseks korpusepõhiseks ühestamiseks tuleb tuttavale korpusepõhisele ühestajale argumendiks anda järjendite järjend, kus iga sisemine järjend sisaldab ühe alamhulga `Text`-objekte:

In [12]:
two_lvl_corpus = [['Esimesele kohale tuleb Jänes, kuigi tema punktide summa pole kõrgeim.',
                   'Lõpparvestuses läks Karule esimene koht. Teise koha sai seekord Jänes. '
                   'Uus võistlus toimub 2. mail.',
                   'Jänesega jokk.'],
                  ['Karu paistis silma suurima punktide summaga. Uue võistluse toimumisajaks '
                   'on 2. mai.',
                   'Karu ja Jänes jäävad ootama 2. maid.']]

In [13]:
# Valmistame tekstid ühestamiseks ette
nested_corpus = []
for articles in two_lvl_corpus:
    article_texts = []
    for t in articles:
        article_texts.append( Text(t).tag_layer(['words','sentences']) ) 
    nested_corpus.append(article_texts)  # lisame korpusse ühe artikli Text-objektid järjendina

In [14]:
vm_corpus_tagger = VabamorfCorpusTagger(output_layer='morph_analysis_with_two_lvl_disamb')
vm_corpus_tagger.tag(nested_corpus)  # märgendame korraga kogu korpuse

[[Text(text='Esimesele kohale tuleb Jänes, kuigi tema punktide summa pole kõrgeim.'),
  Text(text='Lõpparvestuses läks Karule esimene koht. Teise koha sai seekord Jänes. Uus võistlus toimub 2. mail.'),
  Text(text='Jänesega jokk.')],
 [Text(text='Karu paistis silma suurima punktide summaga. Uue võistluse toimumisajaks on 2. mai.'),
  Text(text='Karu ja Jänes jäävad ootama 2. maid.')]]

In [15]:
for article in nested_corpus:
    for article_text in article:
        display(article_text.morph_analysis_with_two_lvl_disamb)

Layer(name='morph_analysis_with_two_lvl_disamb', attributes=('normalized_text', 'lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech'), spans=SL[Span('Esimesele', [{'normalized_text': 'Esimesele', 'lemma': 'esimene', 'root': 'esimene', 'root_tokens': ['esimene'], 'ending': 'le', 'clitic': '', 'form': 'sg all', 'partofspeech': 'O'}]),
Span('kohale', [{'normalized_text': 'kohale', 'lemma': 'koht', 'root': 'koht', 'root_tokens': ['koht'], 'ending': 'le', 'clitic': '', 'form': 'sg all', 'partofspeech': 'S'}]),
Span('tuleb', [{'normalized_text': 'tuleb', 'lemma': 'tulema', 'root': 'tule', 'root_tokens': ['tule'], 'ending': 'b', 'clitic': '', 'form': 'b', 'partofspeech': 'V'}]),
Span('Jänes', [{'normalized_text': 'Jänes', 'lemma': 'Jänes', 'root': 'Jänes', 'root_tokens': ['Jänes'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'H'}]),
Span(',', [{'normalized_text': ',', 'lemma': ',', 'root': ',', 'root_tokens': [','], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z'}]),
Span('kuigi', [{'normalized_text': 'kuigi', 'lemma': 'kuigi', 'root': 'kuigi', 'root_tokens': ['kuigi'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'D'}, {'normalized_text': 'kuigi', 'lemma': 'kuigi', 'root': 'kuigi', 'root_tokens': ['kuigi'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'J'}]),
Span('tema', [{'normalized_text': 'tema', 'lemma': 'tema', 'root': 'tema', 'root_tokens': ['tema'], 'ending': '0', 'clitic': '', 'form': 'sg g', 'partofspeech': 'P'}]),
Span('punktide', [{'normalized_text': 'punktide', 'lemma': 'punkt', 'root': 'punkt', 'root_tokens': ['punkt'], 'ending': 'de', 'clitic': '', 'form': 'pl g', 'partofspeech': 'S'}]),
Span('summa', [{'normalized_text': 'summa', 'lemma': 'summa', 'root': 'summa', 'root_tokens': ['summa'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'S'}]),
Span('pole', [{'normalized_text': 'pole', 'lemma': 'olema', 'root': 'ole', 'root_tokens': ['ole'], 'ending': '0', 'clitic': '', 'form': 'neg o', 'partofspeech': 'V'}]),
Span('kõrgeim', [{'normalized_text': 'kõrgeim', 'lemma': 'kõrgeim', 'root': 'kõrgeim', 'root_tokens': ['kõrgeim'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'U'}]),
Span('.', [{'normalized_text': '.', 'lemma': '.', 'root': '.', 'root_tokens': ['.'], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z'}])])

Layer(name='morph_analysis_with_two_lvl_disamb', attributes=('normalized_text', 'lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech'), spans=SL[Span('Lõpparvestuses', [{'normalized_text': 'Lõpparvestuses', 'lemma': 'lõpparvestus', 'root': 'lõpp_arvestus', 'root_tokens': ['lõpp', 'arvestus'], 'ending': 's', 'clitic': '', 'form': 'sg in', 'partofspeech': 'S'}]),
Span('läks', [{'normalized_text': 'läks', 'lemma': 'minema', 'root': 'mine', 'root_tokens': ['mine'], 'ending': 's', 'clitic': '', 'form': 's', 'partofspeech': 'V'}]),
Span('Karule', [{'normalized_text': 'Karule', 'lemma': 'Karu', 'root': 'Karu', 'root_tokens': ['Karu'], 'ending': 'le', 'clitic': '', 'form': 'sg all', 'partofspeech': 'H'}]),
Span('esimene', [{'normalized_text': 'esimene', 'lemma': 'esimene', 'root': 'esimene', 'root_tokens': ['esimene'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'O'}]),
Span('koht', [{'normalized_text': 'koht', 'lemma': 'koht', 'root': 'koht', 'root_tokens': ['koht'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'S'}]),
Span('.', [{'normalized_text': '.', 'lemma': '.', 'root': '.', 'root_tokens': ['.'], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z'}]),
Span('Teise', [{'normalized_text': 'Teise', 'lemma': 'teine', 'root': 'teine', 'root_tokens': ['teine'], 'ending': '0', 'clitic': '', 'form': 'sg g', 'partofspeech': 'O'}, {'normalized_text': 'Teise', 'lemma': 'teine', 'root': 'teine', 'root_tokens': ['teine'], 'ending': '0', 'clitic': '', 'form': 'sg g', 'partofspeech': 'P'}]),
Span('koha', [{'normalized_text': 'koha', 'lemma': 'koht', 'root': 'koht', 'root_tokens': ['koht'], 'ending': '0', 'clitic': '', 'form': 'sg g', 'partofspeech': 'S'}]),
Span('sai', [{'normalized_text': 'sai', 'lemma': 'saama', 'root': 'saa', 'root_tokens': ['saa'], 'ending': 'i', 'clitic': '', 'form': 's', 'partofspeech': 'V'}]),
Span('seekord', [{'normalized_text': 'seekord', 'lemma': 'seekord', 'root': 'see_kord', 'root_tokens': ['see', 'kord'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'D'}]),
Span('Jänes', [{'normalized_text': 'Jänes', 'lemma': 'Jänes', 'root': 'Jänes', 'root_tokens': ['Jänes'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'H'}]),
Span('.', [{'normalized_text': '.', 'lemma': '.', 'root': '.', 'root_tokens': ['.'], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z'}]),
Span('Uus', [{'normalized_text': 'Uus', 'lemma': 'uus', 'root': 'uus', 'root_tokens': ['uus'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'A'}]),
Span('võistlus', [{'normalized_text': 'võistlus', 'lemma': 'võistlus', 'root': 'võistlus', 'root_tokens': ['võistlus'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'S'}]),
Span('toimub', [{'normalized_text': 'toimub', 'lemma': 'toimuma', 'root': 'toimu', 'root_tokens': ['toimu'], 'ending': 'b', 'clitic': '', 'form': 'b', 'partofspeech': 'V'}]),
Span('2.', [{'normalized_text': '2.', 'lemma': '2.', 'root': '2.', 'root_tokens': ['2.'], 'ending': '0', 'clitic': '', 'form': '?', 'partofspeech': 'O'}]),
Span('mail', [{'normalized_text': 'mail', 'lemma': 'mai', 'root': 'mai', 'root_tokens': ['mai'], 'ending': 'l', 'clitic': '', 'form': 'sg ad', 'partofspeech': 'S'}]),
Span('.', [{'normalized_text': '.', 'lemma': '.', 'root': '.', 'root_tokens': ['.'], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z'}])])

Layer(name='morph_analysis_with_two_lvl_disamb', attributes=('normalized_text', 'lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech'), spans=SL[Span('Jänesega', [{'normalized_text': 'Jänesega', 'lemma': 'Jänes', 'root': 'Jänes', 'root_tokens': ['Jänes'], 'ending': 'ga', 'clitic': '', 'form': 'sg kom', 'partofspeech': 'H'}]),
Span('jokk', [{'normalized_text': 'jokk', 'lemma': 'jokk', 'root': 'jokk', 'root_tokens': ['jokk'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'D'}]),
Span('.', [{'normalized_text': '.', 'lemma': '.', 'root': '.', 'root_tokens': ['.'], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z'}])])

Layer(name='morph_analysis_with_two_lvl_disamb', attributes=('normalized_text', 'lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech'), spans=SL[Span('Karu', [{'normalized_text': 'Karu', 'lemma': 'Karu', 'root': 'Karu', 'root_tokens': ['Karu'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'H'}]),
Span('paistis', [{'normalized_text': 'paistis', 'lemma': 'paistma', 'root': 'paist', 'root_tokens': ['paist'], 'ending': 'is', 'clitic': '', 'form': 's', 'partofspeech': 'V'}]),
Span('silma', [{'normalized_text': 'silma', 'lemma': 'silm', 'root': 'silm', 'root_tokens': ['silm'], 'ending': '0', 'clitic': '', 'form': 'adt', 'partofspeech': 'S'}]),
Span('suurima', [{'normalized_text': 'suurima', 'lemma': 'suurim', 'root': 'suurim', 'root_tokens': ['suurim'], 'ending': '0', 'clitic': '', 'form': 'sg g', 'partofspeech': 'U'}]),
Span('punktide', [{'normalized_text': 'punktide', 'lemma': 'punkt', 'root': 'punkt', 'root_tokens': ['punkt'], 'ending': 'de', 'clitic': '', 'form': 'pl g', 'partofspeech': 'S'}]),
Span('summaga', [{'normalized_text': 'summaga', 'lemma': 'summa', 'root': 'summa', 'root_tokens': ['summa'], 'ending': 'ga', 'clitic': '', 'form': 'sg kom', 'partofspeech': 'S'}]),
Span('.', [{'normalized_text': '.', 'lemma': '.', 'root': '.', 'root_tokens': ['.'], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z'}]),
Span('Uue', [{'normalized_text': 'Uue', 'lemma': 'uus', 'root': 'uus', 'root_tokens': ['uus'], 'ending': '0', 'clitic': '', 'form': 'sg g', 'partofspeech': 'A'}]),
Span('võistluse', [{'normalized_text': 'võistluse', 'lemma': 'võistlus', 'root': 'võistlus', 'root_tokens': ['võistlus'], 'ending': '0', 'clitic': '', 'form': 'sg g', 'partofspeech': 'S'}]),
Span('toimumisajaks', [{'normalized_text': 'toimumisajaks', 'lemma': 'toimumisaeg', 'root': 'toimumis_aeg', 'root_tokens': ['toimumis', 'aeg'], 'ending': 'ks', 'clitic': '', 'form': 'sg tr', 'partofspeech': 'S'}]),
Span('on', [{'normalized_text': 'on', 'lemma': 'olema', 'root': 'ole', 'root_tokens': ['ole'], 'ending': '0', 'clitic': '', 'form': 'b', 'partofspeech': 'V'}, {'normalized_text': 'on', 'lemma': 'olema', 'root': 'ole', 'root_tokens': ['ole'], 'ending': '0', 'clitic': '', 'form': 'vad', 'partofspeech': 'V'}]),
Span('2.', [{'normalized_text': '2.', 'lemma': '2.', 'root': '2.', 'root_tokens': ['2.'], 'ending': '0', 'clitic': '', 'form': '?', 'partofspeech': 'O'}]),
Span('mai', [{'normalized_text': 'mai', 'lemma': 'mai', 'root': 'mai', 'root_tokens': ['mai'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'S'}]),
Span('.', [{'normalized_text': '.', 'lemma': '.', 'root': '.', 'root_tokens': ['.'], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z'}])])

Layer(name='morph_analysis_with_two_lvl_disamb', attributes=('normalized_text', 'lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech'), spans=SL[Span('Karu', [{'normalized_text': 'Karu', 'lemma': 'Karu', 'root': 'Karu', 'root_tokens': ['Karu'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'H'}]),
Span('ja', [{'normalized_text': 'ja', 'lemma': 'ja', 'root': 'ja', 'root_tokens': ['ja'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'J'}]),
Span('Jänes', [{'normalized_text': 'Jänes', 'lemma': 'Jänes', 'root': 'Jänes', 'root_tokens': ['Jänes'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'H'}]),
Span('jäävad', [{'normalized_text': 'jäävad', 'lemma': 'jääma', 'root': 'jää', 'root_tokens': ['jää'], 'ending': 'vad', 'clitic': '', 'form': 'vad', 'partofspeech': 'V'}]),
Span('ootama', [{'normalized_text': 'ootama', 'lemma': 'ootama', 'root': 'oota', 'root_tokens': ['oota'], 'ending': 'ma', 'clitic': '', 'form': 'ma', 'partofspeech': 'V'}]),
Span('2.', [{'normalized_text': '2.', 'lemma': '2.', 'root': '2.', 'root_tokens': ['2.'], 'ending': '0', 'clitic': '', 'form': '?', 'partofspeech': 'O'}]),
Span('maid', [{'normalized_text': 'maid', 'lemma': 'mai', 'root': 'mai', 'root_tokens': ['mai'], 'ending': 'd', 'clitic': '', 'form': 'sg p', 'partofspeech': 'S'}]),
Span('.', [{'normalized_text': '.', 'lemma': '.', 'root': '.', 'root_tokens': ['.'], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z'}])])

Korpusepõhise ühestamise teooriat kirjeldab [see artikkel](https://www.etis.ee/File/DownloadPublic/ce4606c7-41ba-4059-ac6d-6db2e40442e9?name=Fail_FAIA247-0082.pdf&type=application%2Fpdf). EstNLTK korpusepõhise ühestaja liidest kirjeldab detailsemalt [see abimaterjal](https://github.com/estnltk/estnltk/blob/version_1.6/tutorials/nlp_pipeline/B_07b_morph_analysis_with_corpus-based_disambiguation.ipynb).

### Ülesanne 1: Kui palju vähendab korpusepõhine ühestamine mitmesust? (1,25p)

Esimeseks ülesandeks on uurida, kui palju vähendab **korpusepõhine ühestamine** mitmesust. Kataloogis `'aja_sloleht_1999_04_k'` on SL Õhtulehe ajaleheartiklid. 
Igas failis on üks artikkel. 
Lugege sealt tekstid sisse ning teostage morfoloogiline ühestamine kahel erineval viisil: 1) tavaline, nn lausepõhine ühestamine, 2) korpusepõhine ühestamine, kus korpuseks on kõik artiklid, st terve kataloogi sisu.
Leidke ja väljastage mõlema ühestamisetapi kohta mitmeseks jäänud sõnade arv ja osakaal kõigist sõnadest (%).

Lisajuhis:

 * Failide kõvakettalt sisselugemine on üldiselt ressurssinõudlik operatsioon; seetõttu on soovitav ülesanne lahendada nii, et loete kõigi failide sisud kõigepealt mällu (nt järjendisse) ja lahendate alamülesanded 1) ja 2) mälus olevatel andmetel;

### Ülesanne 2: Pärisnimede ühestamise kvaliteedi uurimine (1,75 p)

Korpuste kasutajaid huvitab sageli pärisnimede otsimine. 
Järgnev ülesanne on uurida, kas ja kuidas aitab **tekstipõhine ühestamine** kaasa pärisnimeanalüüside kvaliteedi paranemisele.
Looge programm, mis analüüsib korpust kataloogis `'aja_sloleht_1999_04_k'` ning väljastab kõik laused, kus leidub sõnu, mille puhul tavaline ühestamine ei andnud üldse pärisnime analüüsi, aga tekstipõhise ühestamise järel sai sõna ühese pärisnime analüüsi. Programm väljastab lausete tekstid, ühese pärisnimeanalüüsi saanud sõnad ja nende analüüside muutuse. Näide:

    >> aja_sloleht_1999_04_15__20.txt
    Matkamiksi peakorraldaja Andrus Nõmm tunnistas , et turvafirma Desperada kuuele töötajale ei ole tal midagi ette heita .
     Nõmm
       [('nõmm', 'S', 'sg n')]
       ==>
       [('Nõmm', 'H', 'sg n')]

    “ Olin ka ise seal , ” rääkis Nõmm .
     Nõmm
       [('nõmm', 'S', 'sg n')]
       ==>
       [('Nõmm', 'H', 'sg n')]

    Viha noorukite peale Kare ei pea , sest kaupluse alkoholikäive suurened messi ajal mitu korda .
     Kare
       [('kare', 'A', 'sg n')]
       ==>
       [('Kare', 'H', 'sg n')]
    
Kuidas sellele ülesandele võiks läheneda? Esmalt leida igast lausest sõnad, mis said (tekstipõhise) ühestamise tulemusena ühese pärisnime analüüsi (sõnaliik: `'H'`) ning võrrelda seda tavalise ühestamise väljundiga (eelmise ülesande ühestamine 1). Leidke sobiva muutusega sõnad.
Väljastage laused, kuhu tekkisid ühese pärisnimeanalüüsiga sõnad, ning vastavad sõnad ja nende analüüside muutused (väljund võiks olla eeltoodud näitega sarnane).

Veel näpunäiteid:

  * Kui teete tsükli, mis käib üle korpuse kõigi tekstide, arvestage sellega, et korpus on üksjagu mahukas. Alguses on katsetamiseks soovitav teha kõigepealt tsükkel, mis katkestatakse [`break`](https://docs.python.org/3.5/tutorial/controlflow.html#break-and-continue-statements-and-else-clauses-on-loops) käsuga, kui teatud arv esimesi tekste (või lauseid) on juba läbitud. Kui olete jõudnud niikaugele, et programm töötab ilusti väiksel osal korpusest, siis saab ka katkestusest loobuda ja panna tsükli(d) jooksma üle kogu korpuse;

## Morfoloogiline analüüs ja ühestamine kasutajasõnastiku abil

Morfoloogiline analüsaator töötab kõige paremini tekstidel, mis järgivad kirjakeele norme. Kuigi analüsaator sisaldab ka tundmatute sõnade oletajat, ei saa selle täpsusele lootma jääda, kui analüüsitavad tekstid erinevad kirjakeelest suurel määral. Üks võimalus, kuidas analüüsi täpsemaks teha on morfoloogilise analüüsi lekskioni laiendamine kõnekeelega. Seda saame teha make_resolveriga nagu ühestamist ja oletamistki, määrates parameetri `slang_lex`:

In [16]:
from estnltk.resolve_layer_dag import make_resolver

resolver = make_resolver(slang_lex=True)

In [17]:
sent = "Mul on veits halb olla täna, muideks. Tahax ibukat."

# Loome tavalise morf. analüüsiga teksti
sent_regular = Text(sent).tag_layer() 

# Slängiga laiendatud morf. analüüs
sent_slang = Text(sent).tag_layer(resolver=resolver)

In [18]:
display(sent_regular.morph_analysis)

Layer(name='morph_analysis', attributes=('normalized_text', 'lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech'), spans=SL[Span('Mul', [{'normalized_text': 'Mul', 'lemma': 'mina', 'root': 'mina', 'root_tokens': ['mina'], 'ending': 'l', 'clitic': '', 'form': 'sg ad', 'partofspeech': 'P'}]),
Span('on', [{'normalized_text': 'on', 'lemma': 'olema', 'root': 'ole', 'root_tokens': ['ole'], 'ending': '0', 'clitic': '', 'form': 'b', 'partofspeech': 'V'}, {'normalized_text': 'on', 'lemma': 'olema', 'root': 'ole', 'root_tokens': ['ole'], 'ending': '0', 'clitic': '', 'form': 'vad', 'partofspeech': 'V'}]),
Span('veits', [{'normalized_text': 'veits', 'lemma': 'veits', 'root': 'veits', 'root_tokens': ['veits'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'S'}]),
Span('halb', [{'normalized_text': 'halb', 'lemma': 'halb', 'root': 'halb', 'root_tokens': ['halb'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'A'}]),
Span('olla', [{'normalized_text': 'olla', 'lemma': 'olema', 'root': 'ole', 'root_tokens': ['ole'], 'ending': 'a', 'clitic': '', 'form': 'da', 'partofspeech': 'V'}]),
Span('täna', [{'normalized_text': 'täna', 'lemma': 'täna', 'root': 'täna', 'root_tokens': ['täna'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'D'}]),
Span(',', [{'normalized_text': ',', 'lemma': ',', 'root': ',', 'root_tokens': [','], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z'}]),
Span('muideks', [{'normalized_text': 'muideks', 'lemma': 'mui', 'root': 'mui', 'root_tokens': ['mui'], 'ending': 'deks', 'clitic': '', 'form': 'pl tr', 'partofspeech': 'S'}]),
Span('.', [{'normalized_text': '.', 'lemma': '.', 'root': '.', 'root_tokens': ['.'], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z'}]),
Span('Tahax', [{'normalized_text': 'Tahax', 'lemma': 'Tahax', 'root': 'Tahax', 'root_tokens': ['Tahax'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'H'}]),
Span('ibukat', [{'normalized_text': 'ibukat', 'lemma': 'ibukat', 'root': 'ibukat', 'root_tokens': ['ibukat'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'S'}]),
Span('.', [{'normalized_text': '.', 'lemma': '.', 'root': '.', 'root_tokens': ['.'], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z'}])])

Tavalise morfoloogilise analüüsi puhul on saanud osaliselt või täiesti vale märgenduse neli sõna: *veits*, *muideks*, *Tahax*, *ibukat*.

Vaatame laiendatud leksikoniga analüüsi:

In [19]:
display(sent_slang.morph_analysis)

Layer(name='morph_analysis', attributes=('normalized_text', 'lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech'), spans=SL[Span('Mul', [{'normalized_text': 'Mul', 'lemma': 'mina', 'root': 'mina', 'root_tokens': ['mina'], 'ending': 'l', 'clitic': '', 'form': 'sg ad', 'partofspeech': 'P'}]),
Span('on', [{'normalized_text': 'on', 'lemma': 'olema', 'root': 'ole', 'root_tokens': ['ole'], 'ending': '0', 'clitic': '', 'form': 'b', 'partofspeech': 'V'}, {'normalized_text': 'on', 'lemma': 'olema', 'root': 'ole', 'root_tokens': ['ole'], 'ending': '0', 'clitic': '', 'form': 'vad', 'partofspeech': 'V'}]),
Span('veits', [{'normalized_text': 'veits', 'lemma': 'veits', 'root': 'veits', 'root_tokens': ['veits'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'D'}]),
Span('halb', [{'normalized_text': 'halb', 'lemma': 'halb', 'root': 'halb', 'root_tokens': ['halb'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'A'}]),
Span('olla', [{'normalized_text': 'olla', 'lemma': 'olema', 'root': 'ole', 'root_tokens': ['ole'], 'ending': 'a', 'clitic': '', 'form': 'da', 'partofspeech': 'V'}]),
Span('täna', [{'normalized_text': 'täna', 'lemma': 'täna', 'root': 'täna', 'root_tokens': ['täna'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'D'}]),
Span(',', [{'normalized_text': ',', 'lemma': ',', 'root': ',', 'root_tokens': [','], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z'}]),
Span('muideks', [{'normalized_text': 'muideks', 'lemma': 'muideks', 'root': 'muideks', 'root_tokens': ['muideks'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'D'}]),
Span('.', [{'normalized_text': '.', 'lemma': '.', 'root': '.', 'root_tokens': ['.'], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z'}]),
Span('Tahax', [{'normalized_text': 'Tahax', 'lemma': 'Tahax', 'root': 'Tahax', 'root_tokens': ['Tahax'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'H'}]),
Span('ibukat', [{'normalized_text': 'ibukat', 'lemma': 'ibukat', 'root': 'ibukat', 'root_tokens': ['ibukat'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'S'}]),
Span('.', [{'normalized_text': '.', 'lemma': '.', 'root': '.', 'root_tokens': ['.'], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z'}])])

Nagu selgub ka, ei piisa alati ainult slang_lexi määramisest, et tulemused oleksid rahuldavad. Seetõttu kasutatakse netikeele vms eesti keele allkeele automaatseks analüüsimiseks nn _kasutajasõnastiku lähenemist_. Selleks on EstNLTKs klass `UserDictTagger`, mis võimaldab kasutajasõnastikku ühekaupa sõnu ja analüüse lisada või need CSV-failist laadida. `UserDictTagger` märgendab üle `morph_analysis` kihi.

Vaatame seda protsessi samm-sammult netikeele lausete näitel.

Kasutajasõnastiku abil saab märgendusi muuta kahel moel: märgendusi osaliselt üle kirjutades ja märgendusi täielikult üle kirjutades. Vaatame kõigepealt **osalist ülekirjutamist**.

Esmalt märgendame teksti vaikimise seadetega:

In [20]:
text = Text('kas keegi juttu ei viitci ajada ?').tag_layer()
text.morph_analysis

Layer(name='morph_analysis', attributes=('normalized_text', 'lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech'), spans=SL[Span('kas', [{'normalized_text': 'kas', 'lemma': 'kas', 'root': 'kas', 'root_tokens': ['kas'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'D'}]),
Span('keegi', [{'normalized_text': 'keegi', 'lemma': 'keegi', 'root': 'keegi', 'root_tokens': ['keegi'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'P'}]),
Span('juttu', [{'normalized_text': 'juttu', 'lemma': 'jutt', 'root': 'jutt', 'root_tokens': ['jutt'], 'ending': '0', 'clitic': '', 'form': 'sg p', 'partofspeech': 'S'}]),
Span('ei', [{'normalized_text': 'ei', 'lemma': 'ei', 'root': 'ei', 'root_tokens': ['ei'], 'ending': '0', 'clitic': '', 'form': 'neg', 'partofspeech': 'V'}]),
Span('viitci', [{'normalized_text': 'viitci', 'lemma': 'viitcima', 'root': 'viitci', 'root_tokens': ['viitci'], 'ending': '0', 'clitic': '', 'form': 'o', 'partofspeech': 'V'}]),
Span('ajada', [{'normalized_text': 'ajada', 'lemma': 'ajama', 'root': 'aja', 'root_tokens': ['aja'], 'ending': 'da', 'clitic': '', 'form': 'da', 'partofspeech': 'V'}]),
Span('?', [{'normalized_text': '?', 'lemma': '?', 'root': '?', 'root_tokens': ['?'], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z'}])])

Sõna *viitci* märgendus oli osaliselt korrektne: sõnaliik, vorm ja lõpp on korrektsed. Siiski tahaks, et lemma järgiks kirjakeele normi. Impordime kasutajasõnastiku loomiseks klassi `UserDictTagger` ja loome selle objekti

In [21]:
from estnltk.taggers import UserDictTagger
userdict = UserDictTagger(ignore_case=True)  # vaikimisi suurtähetundlik

Analüüsi osaliselt muutmiseks peab olema ülekirjutatud vähemalt üks väljadest `root`, `ending`, `clitic`, `form` või `partofspeech`. Kui juur `root` on defineeritud, peab märkima ka sõnaliigi. Nende põhjal saavad ka atribuudid `lemma` ja `root_tokens` uued väärtused, ilma et neid eraldi täpsustaks.

In [22]:
userdict.add_word('viitci', {'root': 'viitsi', 'partofspeech': 'V'})

Parandame anlüüsid ja vaatame muudatust:

In [23]:
userdict.retag(text)
text.morph_analysis[4]

text,normalized_text,lemma,root,root_tokens,ending,clitic,form,partofspeech
viitci,viitci,viitsima,viitsi,['viitsi'],0,,o,V


Analüüside **täielikku ülekirjutamist** alustame samuti morfoloogilisest analüüsist.

In [24]:
sent = "Sa ajad sássi inimmeste erinevad käsitlusviisid ja lóodusnähhtuste kinndla vahekorra." 
text = Text(sent).tag_layer(['morph_analysis'])
text.morph_analysis

Layer(name='morph_analysis', attributes=('normalized_text', 'lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech'), spans=SL[Span('Sa', [{'normalized_text': 'Sa', 'lemma': 'sina', 'root': 'sina', 'root_tokens': ['sina'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'P'}]),
Span('ajad', [{'normalized_text': 'ajad', 'lemma': 'aeg', 'root': 'aeg', 'root_tokens': ['aeg'], 'ending': 'd', 'clitic': '', 'form': 'pl n', 'partofspeech': 'S'}]),
Span('sássi', [{'normalized_text': 'sássi', 'lemma': 'sáss', 'root': 'sáss', 'root_tokens': ['sáss'], 'ending': '0', 'clitic': '', 'form': 'sg g', 'partofspeech': 'S'}, {'normalized_text': 'sássi', 'lemma': 'sássi', 'root': 'sássi', 'root_tokens': ['sássi'], 'ending': '0', 'clitic': '', 'form': 'sg g', 'partofspeech': 'S'}]),
Span('inimmeste', [{'normalized_text': 'inimmeste', 'lemma': 'inimmest', 'root': 'inim_mest', 'root_tokens': ['inim', 'mest'], 'ending': 'e', 'clitic': '', 'form': 'pl p', 'partofspeech': 'S'}]),
Span('erinevad', [{'normalized_text': 'erinevad', 'lemma': 'erinev', 'root': 'erinev', 'root_tokens': ['erinev'], 'ending': 'd', 'clitic': '', 'form': 'pl n', 'partofspeech': 'A'}]),
Span('käsitlusviisid', [{'normalized_text': 'käsitlusviisid', 'lemma': 'käsitlusviis', 'root': 'käsitlus_viis', 'root_tokens': ['käsitlus', 'viis'], 'ending': 'd', 'clitic': '', 'form': 'pl n', 'partofspeech': 'S'}]),
Span('ja', [{'normalized_text': 'ja', 'lemma': 'ja', 'root': 'ja', 'root_tokens': ['ja'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'J'}]),
Span('lóodusnähhtuste', [{'normalized_text': 'lóodusnähhtuste', 'lemma': 'lóodusnähhtus', 'root': 'lóodusnähhtus', 'root_tokens': ['lóodusnähhtus'], 'ending': 'te', 'clitic': '', 'form': 'pl g', 'partofspeech': 'S'}, {'normalized_text': 'lóodusnähhtuste', 'lemma': 'lóodusnähhtuste', 'root': 'lóodusnähhtuste', 'root_tokens': ['lóodusnähhtuste'], 'ending': '0', 'clitic': '', 'form': 'sg g', 'partofspeech': 'S'}]),
Span('kinndla', [{'normalized_text': 'kinndla', 'lemma': 'kinndla', 'root': 'kinndla', 'root_tokens': ['kinndla'], 'ending': '0', 'clitic': '', 'form': 'sg g', 'partofspeech': 'S'}]),
Span('vahekorra', [{'normalized_text': 'vahekorra', 'lemma': 'vahekord', 'root': 'vahe_kord', 'root_tokens': ['vahe', 'kord'], 'ending': '0', 'clitic': '', 'form': 'sg g', 'partofspeech': 'S'}]),
Span('.', [{'normalized_text': '.', 'lemma': '.', 'root': '.', 'root_tokens': ['.'], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z'}])])

Meie näide on lihtne selles mõttes, et kuigi siin on mitte-kirjakeelseid sõnavorme (nt *inimmeste*, *lóodusnähhtuste*), leidub kõigile sõnadele siiski kirjakeeles vaste. Seega saame tekitada mitte-kirjakeelsete vormide morfoloogilised analüüsid, kui analüüsime vastavaid kirjakeele sõnavorme:

In [25]:
# Loome uue kasutajasõnastiku
userdict = UserDictTagger( ignore_case=True )
# Lisame tundmatute sõnade vormid koos neile vastavate kirjakeele sõnade analüüsidega
userdict.add_word('inimmeste', [{'normalized_text': 'inimeste', 'form': 'pl g', 'root': 'inimene', 'ending': 'te', 'partofspeech': 'S', 'clitic': ''}])
userdict.add_word('kinndla', [{'normalized_text': 'kindla', 'form': 'sg g', 'root': 'kindel', 'ending': '0', 'partofspeech': 'A', 'clitic': ''}])
userdict.add_word('lóodusnähhtuste', [{'normalized_text': 'loodusnähtuste', 'form': 'pl g', 'root': 'loodus_nähtus', 'ending': 'te', 'partofspeech': 'S', 'clitic': ''}])

In [26]:
userdict.retag( text )
text.morph_analysis

Layer(name='morph_analysis', attributes=('normalized_text', 'lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech'), spans=SL[Span('Sa', [{'normalized_text': 'Sa', 'lemma': 'sina', 'root': 'sina', 'root_tokens': ['sina'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'P'}]),
Span('ajad', [{'normalized_text': 'ajad', 'lemma': 'aeg', 'root': 'aeg', 'root_tokens': ['aeg'], 'ending': 'd', 'clitic': '', 'form': 'pl n', 'partofspeech': 'S'}]),
Span('sássi', [{'normalized_text': 'sássi', 'lemma': 'sáss', 'root': 'sáss', 'root_tokens': ['sáss'], 'ending': '0', 'clitic': '', 'form': 'sg g', 'partofspeech': 'S'}, {'normalized_text': 'sássi', 'lemma': 'sássi', 'root': 'sássi', 'root_tokens': ['sássi'], 'ending': '0', 'clitic': '', 'form': 'sg g', 'partofspeech': 'S'}]),
Span('inimmeste', [{'normalized_text': 'inimeste', 'lemma': 'inimene', 'root': 'inimene', 'root_tokens': ['inimene'], 'ending': 'te', 'clitic': '', 'form': 'pl g', 'partofspeech': 'S'}]),
Span('erinevad', [{'normalized_text': 'erinevad', 'lemma': 'erinev', 'root': 'erinev', 'root_tokens': ['erinev'], 'ending': 'd', 'clitic': '', 'form': 'pl n', 'partofspeech': 'A'}]),
Span('käsitlusviisid', [{'normalized_text': 'käsitlusviisid', 'lemma': 'käsitlusviis', 'root': 'käsitlus_viis', 'root_tokens': ['käsitlus', 'viis'], 'ending': 'd', 'clitic': '', 'form': 'pl n', 'partofspeech': 'S'}]),
Span('ja', [{'normalized_text': 'ja', 'lemma': 'ja', 'root': 'ja', 'root_tokens': ['ja'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'J'}]),
Span('lóodusnähhtuste', [{'normalized_text': 'loodusnähtuste', 'lemma': 'loodusnähtus', 'root': 'loodus_nähtus', 'root_tokens': ['loodus', 'nähtus'], 'ending': 'te', 'clitic': '', 'form': 'pl g', 'partofspeech': 'S'}]),
Span('kinndla', [{'normalized_text': 'kindla', 'lemma': 'kindel', 'root': 'kindel', 'root_tokens': ['kindel'], 'ending': '0', 'clitic': '', 'form': 'sg g', 'partofspeech': 'A'}]),
Span('vahekorra', [{'normalized_text': 'vahekorra', 'lemma': 'vahekord', 'root': 'vahe_kord', 'root_tokens': ['vahe', 'kord'], 'ending': '0', 'clitic': '', 'form': 'sg g', 'partofspeech': 'S'}]),
Span('.', [{'normalized_text': '.', 'lemma': '.', 'root': '.', 'root_tokens': ['.'], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z'}])])

Nagu näha *lóodusnähhtuste* näitel, kirjutab sellisel moel analüüside lisamine üle kõik olemasolevad analüüsid. Atribuudid `root`, `ending`, `clitic`, `form` ja `partofspeech` peavad analüüside järjendina ette andmisel kindlasti täpsustatud olema. Ette saab anda ka mitu võimalikku analüüsi.

Sõnal *sassi* võib kirjakeeles olla kaks analüüsi, lisame mõlemad kasutajasõnastikku *sássi* vasteks

In [27]:
userdict.add_word('sássi', [{'normalized_text': 'sassi', 'form': '', 'root': 'sassi', 'ending': '0', 'partofspeech': 'D', 'clitic': ''},
                           {'normalized_text': 'sassi', 'form': 'adt', 'root': 'sasi', 'ending': '0', 'partofspeech': 'S', 'clitic': ''}])


In [28]:
userdict.retag(text)
text.morph_analysis[2]

text,normalized_text,lemma,root,root_tokens,ending,clitic,form,partofspeech
sássi,sassi,sassi,sassi,['sassi'],0,,,D
,sassi,sasi,sasi,['sasi'],0,,adt,S


Vaadeldavas lauses on siiski õige vaid üks kahest analüüsist.
Impordime Vabamorfi lausepõhise ühestaja, et saada ühestatud `morph_analysis` kiht uute tulemuste põhjal.

In [29]:
from estnltk.taggers import VabamorfDisambiguator

vm_disambiguator = VabamorfDisambiguator()
vm_disambiguator.retag(text)
text.morph_analysis[2]

text,normalized_text,lemma,root,root_tokens,ending,clitic,form,partofspeech
sássi,sassi,sassi,sassi,['sassi'],0,,,D


## Kasutajasõnastiku (pool-)automaatne loomine

Suure korpuse korral on kasutajasõnastiku loomine siiski väga töömahukas tegevus ning täieliku käsitööna mitte eriti mõeldav. Parem lähenemine on kirjakeelsete vastete leidmine vähemalt osaliselt automatiseerida. Järgnevates ülesannetes uurimegi üht sellise automatiseerimisprotsessi alamosa: tundmatutele sõnadele kirjakeelsete vastete leidmist. Kõigepealt peame aga kindlaks tegema, millised sõnad saab üldse tundmatuteks lugeda (arvestades uuritava korpuse sõnestuse eripärasid).

### Ülesanne 3: Tundmatud sõnad (1,5 p)

Looge programm, mis loeb failist `'netikeele_laused.txt'` laused (failis on iga lause eraldi real) ning teostab lausetel morf analüüsi (ilma oletamise, pärisnimede pakkumise ja ühestamiseta). 

Programm leiab: 
* mitu protsenti kõigist sõnadest jäid morf analüsaatorile tundmatuks (st sõnadele ei leitud analüüse)
* 10 lauset, mis sisaldavad kõige rohkem tundmatuid sõnu.

Leitud 10 lauset ja nendes tundmatuks jäänud sõnad väljastatakse ekraanile ning ühtlasi salvestatakse ka kuhugi andmestruktuuri – neid läheb tarvis ka järgmises ülesandes. Ekraanile väljastamisel peavad laused olema sorteeritud "tundmatute arvu järgi" kahanevalt.

  * Sõnade loendamisel tuleks välja jätta 1-tähelised "sõnad", ainult numbritest ja punktuatsioonist koosnevad sõnad (st näiteks 20-aastane on korrektne sõna);
  * "Tundmatute sõnade" hulka ei tuleks lugeda punktuatsiooni;
  * 10 kõige rohkem tundmatuid sõnu sisaldava lause hulka võib sattuda ka lauseid, milles on sama arv tundmatuid sõnu. Kui viimas(t)ele positsiooni(de)le on valida mitme sama arvu tundmatuid sisaldava lause vahel, siis võib valida lause(d), mis on algses (failis olevas) järjestuses eespool;

### Teisenduskauguse abil vastete leidmine

Nüüd on meil teada, millist laadi sõnadele tuleb hakata kirjakeelseid vasteid otsima. 
Aga kuidas vasteid leida?
Tehniliselt kõige lihtsam lähenemine on kasutada juba valmislahendust – proovida eelmises praktikumis tutvustatud automaatset õigekirjakorrektorit, mida võib ka rakendada tundmatutele sõnadele vastete leidmiseks.
Siiski ei pruugi õigekirjakorrektor kõige paremaid tulemusi anda netikeele kõige konarlikemates lausetes, kus esineb eri tüüpi vigu ning nii mõnigi kord on tegemist süstemaatilise "teisiti kirjutamisega". 

Üldine meetod, mida rakendab (küll teatavate lisanüanssidega) ka õigekirjakorrektor, on tundmatute sõnade võrdlemine teadaolevate sõnade leksikoniga ning sõnade _sarnasuse_ põhjal vastete väljapakkumine. 
Seega, kui meil on olemas piisavalt mahukas kirjakeele korpus, võime selle põhjal ka ise teha teadaolevate sõnade loendi, noppida loendist välja tundmatute sõnadega _sarnased_ sõnad ja pakkuda neid vasteteks.

_Kuidas hinnata sõnade sarnasust?_ Siin tuleb appi teisenduskaugus (ingl *edit distance*). Sisuliselt mõõdab teisenduskaugus, mitu teisendust/muutust tuleb teha, et saada ühest sõnest teine. Tavaliselt lubatakse kolme tüüpi muutuseid: 1) ühe tähe kustutamine, 2) ühe tähe lisamine, 3) ühe tähe asendamine mingi teise tähega. Teisenduskaugus ütleb, mitu sellist muutust tuleb minimaalselt teha – mida väiksem arv, seda sarnasemad kaks sõna on.

Teisenduskauguse leidmist võib katsetada funktsiooni `nltk.metrics.distance.edit_distance` abil (funktsiooni sisaldav teek `nltk` peaks teil olema juba installitud, kuna see on üks `estnltk` eelduseid):

In [30]:
from nltk.metrics.distance import edit_distance

edit_distance('pizza', 'pitsa')

2

In [31]:
edit_distance('mõttetu', 'mõtetu')

1

In [32]:
edit_distance('uit', 'uit')

0

Vt ka `nltk` teisenduskauguse funktsiooni [detailsemat kirjeldust](http://www.nltk.org/api/nltk.metrics.html#nltk.metrics.distance.edit_distance).

### Ülesanne 4: Tundmatutele sõnadele vastete leidmine (2 p)

See ülesanne koosneb kahest osast:

#### Ülesanne 4.A: Minu soovitaja (1,25 p)

Looge funktsioon, mis saab sisendiks tundmatu sõna ning tagastab sellele kõige sarnasemad vasted kirjakeelest. 
Funktsioon võiks osata pakkuda natukene rohkem vasteid kui EstNLTK [õigekirjakontrollija](https://nbviewer.jupyter.org/github/estnltk/estnltk/blob/version_1.6/tutorials/nlp_pipeline/B_03_segmentation_words_spelling_normalization.ipynb). 
Seega, esimese sammuna võibki funktsioon kasutada õigekirjakorrektorit vastete leidmiseks, aga kui korrektor midagi pakkuda ei oska, tuleks teisenduskauguse abil ise vasteid otsida. 
Funktsioon tagastab sarnaste sõnade järjendi (ja tühijärjendi, kui vasteid üldse ei leitud).

Teisenduskauguse abil võiks vasteid otsida ajakirjandustekstide sõnavarast (kataloogi `'aja_sloleht_1999_04_k'` tekstidest). 
Kui suures osas ajakirjendustekstide sõnavara tuleks arvestada (mida jätta ja mida võtta), jääb teie otsustada.
Samuti jääb teie otsustada, kui palju ja millisel teisenduskaugusel vasteid tagastatakse (üldine põhimõte on, et mida vähem ja täpsemaid vasteid pakutakse, seda parem).

  * **NB!** Kataloogi 'aja_sloleht_1999_04_k' tekstide sisselugemine ja nende põhjal sõnaloendi moodustamine peaks toimuma ainult üks kord – kindlasti ei tohiks seda teha igal soovitaja-funktsiooni väljakutsel. 


#### Ülesanne 4.B: Soovitaja rakendamine tundmatute tuvastamiseks (0,75 p)

Rakendage oma soovitaja-funktsiooni ülesandes 3 leitud kümne "kõige konarlikuma" lause tundmatutel sõnadel. Väljastage:

   * laused ning iga lause all nii tundmatud sõnad kui ka neile pakutavad vasted;
   * (üle kõigi lausete) mitmele %-le tundmatutest sõnadest pakutakse vasteid;
   * (üle kõigi lausete) milline on keskmine soovituste arv (ehk siis: mitu vastet keskmiselt iga tundmatu sõna kohta soovitatakse);

Vajadusel täiendage oma funktsiooni nii, et vähemalt 90% tundmatutest sõnadest saaksid soovitusi.

### Soovitustest kasutajasõnastikuni

Eelmises ülesandes arendasime ja katsetasime soovitusfunktsiooni 10 lause tundmatutel sõnadel. 
Praktikas on tavaliselt eesmärgiks terve korpuse analüüs, seega on ka ülesanne üksjagu töömahukam. 
Kuidas praktikas kohandada morfoloogiline analüsaator netikeele tekstide analüüsimiseks, on kirjeldatud detailsemalt [selles artiklis](https://www.etis.ee/File/DownloadPublic/ef492abe-ba06-4e86-9fee-a7e5cd44eeb6?name=Fail_ERYa7.07_Muischnek_pp111-127.pdf&type=application%2Fpdf). Üldine põhimõte on järgmine. Kirjakeelest erinevad ja haruldased sõnad peaksid olema kirjakeelsetest vormidest tuletatavad suhteliselt regulaarsete teisenduste abil. Ehk siis leitavad teisenduskauguse või veelgi spetsiifilisemate teisenduste kaudu; nende lisamine kasutajasõnastikku võiks toimuda suhteliselt automaatselt. Samas, kui sõnavorm pole tuletavav kirjakeelsest sõnast regulaarse teisenduse abil, peaks ta olema sageli kasutatav ("et tema tähendus ja funktsioon kasutajatel meeles püsiks") ja seega korpuse sõnasageduste uurimisel silma torkama. Sellised sõnavormid tuleks siis korpuse sagedusloendite uurimisel välja selgitada ja lisada kasutajasõnastikku käsitsi.

## Giellatekno (GT) morfoloogilised kategooriad

Siiani oleme vaadanud, kuidas parandada oma teksti morfoloogilist analüüsi nii, et see etteantud morfoloogiliste kategooriate süsteemis võimalikult korrektselt analüüsitud saaks. Võib aga juhtuda ka nii, et tekst on küll (piisavalt) korrektselt analüüsitud vaikimisi kasutatavate kategooriate järgi, ent see siiski millegipärast ei rahulda teksti töötlejat. Seetõttu on EstNLTK-s lisaks vaikimisi kasutatavale Filosofti süsteemile ka teine morfoloogiliste kategooriate süsteem: Giellatekno (GT).

Giellatekno morfoloogiliste kategooriate süsteemi saab EstNLTK-s kasutada järgnevalt:

In [33]:
from estnltk import Text
from estnltk.taggers import GTMorphConverter

In [34]:
gt_converter = GTMorphConverter()

In [35]:
text = Text('Mis siis, et kuuldavasti teevadki Poola torumehed nii Pariisis kui ka Brüsselis torud reaalselt korda.')

GT morfoloogilise analüüsi tegemiseks peab olema tehtud nii morfoloogiline analüüs, kui ka osalausete märgendus, mida vaikimisi ei tehta. Osalauseanalüüs nõuab sõnade, lausete ja morfanalüüsi kihi olemasolu.

<div class="alert alert-block alert-warning">

**Tehniline vahemärkus**: osalauseanalüüs kasutab java-t. Seega tuleb enne analüüsimist:
* Installida süsteemi [*Java SE Runtime Environment*](https://www.java.com/en/download/win10.jsp);
* Panna java käsk süsteemi keskkonnamuutujasse PATH. Windows-i ja Mac-i puhul tehakse seda tüüpiliselt juba installi käigus, aga kui on siiski tarvis seda käsitsi teha, siis detailsemat abi saab [siit](https://java.com/en/download/help/path.xml);


Kuidas kontrollida, kas java on juba olemas või kas installimine õnnestus? Käsureakäsk `java -version` peaks kuvama infot installitud java versiooni kohta, näiteks midagi taolist:

~~~
java version "14.0.1" 2020-04-14
Java(TM) SE Runtime Environment (build 14.0.1+7)
Java HotSpot(TM) 64-Bit Server VM (build 14.0.1+7, mixed mode, sharing)  
~~~
</div>

In [36]:
# Märgendame tekstile peale vaikimisi toimiva Filosofti analüüsi
text.tag_layer(['clauses'])

Text(text='Mis siis, et kuuldavasti teevadki Poola torumehed nii Pariisis kui ka Brüsselis torud reaalselt korda.')

In [37]:
# Teisendame morfoloogilise analüüsi GT formaati - tulemused on salvestatud kihis gt_morph_analysis
gt_converter.tag(text)

Text(text='Mis siis, et kuuldavasti teevadki Poola torumehed nii Pariisis kui ka Brüsselis torud reaalselt korda.')

In [38]:
text.gt_morph_analysis

Layer(name='gt_morph_analysis', attributes=('normalized_text', 'lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech'), spans=SL[Span('Mis', [{'normalized_text': 'Mis', 'lemma': 'mis', 'root': 'mis', 'root_tokens': ('mis',), 'ending': '0', 'clitic': '', 'form': 'Pl Nom', 'partofspeech': 'P'}, {'normalized_text': 'Mis', 'lemma': 'mis', 'root': 'mis', 'root_tokens': ('mis',), 'ending': '0', 'clitic': '', 'form': 'Sg Nom', 'partofspeech': 'P'}]),
Span('siis', [{'normalized_text': 'siis', 'lemma': 'siis', 'root': 'siis', 'root_tokens': ('siis',), 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'D'}]),
Span(',', [{'normalized_text': ',', 'lemma': ',', 'root': ',', 'root_tokens': (',',), 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z'}]),
Span('et', [{'normalized_text': 'et', 'lemma': 'et', 'root': 'et', 'root_tokens': ('et',), 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'J'}]),
Span('kuuldavasti', [{'normalized_text': 'kuuldavasti', 'lemma': 'kuuldavasti', 'root': 'kuuldavasti', 'root_tokens': ('kuuldavasti',), 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'D'}]),
Span('teevadki', [{'normalized_text': 'teevadki', 'lemma': 'tegema', 'root': 'tege', 'root_tokens': ('tege',), 'ending': 'vad', 'clitic': 'ki', 'form': 'Pers Prs Ind Pl 3 Aff', 'partofspeech': 'V'}]),
Span('Poola', [{'normalized_text': 'Poola', 'lemma': 'Poola', 'root': 'Poola', 'root_tokens': ('Poola',), 'ending': '0', 'clitic': '', 'form': 'Sg Gen', 'partofspeech': 'H'}]),
Span('torumehed', [{'normalized_text': 'torumehed', 'lemma': 'torumees', 'root': 'toru_mees', 'root_tokens': ('toru', 'mees'), 'ending': 'd', 'clitic': '', 'form': 'Pl Nom', 'partofspeech': 'S'}]),
Span('nii', [{'normalized_text': 'nii', 'lemma': 'nii', 'root': 'nii', 'root_tokens': ('nii',), 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'D'}]),
Span('Pariisis', [{'normalized_text': 'Pariisis', 'lemma': 'Pariis', 'root': 'Pariis', 'root_tokens': ('Pariis',), 'ending': 's', 'clitic': '', 'form': 'Sg Ine', 'partofspeech': 'H'}]),
Span('kui', [{'normalized_text': 'kui', 'lemma': 'kui', 'root': 'kui', 'root_tokens': ('kui',), 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'D'}, {'normalized_text': 'kui', 'lemma': 'kui', 'root': 'kui', 'root_tokens': ('kui',), 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'J'}]),
Span('ka', [{'normalized_text': 'ka', 'lemma': 'ka', 'root': 'ka', 'root_tokens': ('ka',), 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'D'}]),
Span('Brüsselis', [{'normalized_text': 'Brüsselis', 'lemma': 'Brüssel', 'root': 'Brüssel', 'root_tokens': ('Brüssel',), 'ending': 's', 'clitic': '', 'form': 'Sg Ine', 'partofspeech': 'H'}]),
Span('torud', [{'normalized_text': 'torud', 'lemma': 'toru', 'root': 'toru', 'root_tokens': ('toru',), 'ending': 'd', 'clitic': '', 'form': 'Pl Nom', 'partofspeech': 'S'}]),
Span('reaalselt', [{'normalized_text': 'reaalselt', 'lemma': 'reaalselt', 'root': 'reaalselt', 'root_tokens': ('reaalselt',), 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'D'}]),
Span('korda', [{'normalized_text': 'korda', 'lemma': 'kord', 'root': 'kord', 'root_tokens': ('kord',), 'ending': '0', 'clitic': '', 'form': 'Sg Par', 'partofspeech': 'S'}]),
Span('.', [{'normalized_text': '.', 'lemma': '.', 'root': '.', 'root_tokens': ('.',), 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z'}])])

### Boonusülesanne (0,5p)

Tutvuge giellatekno formaadis morfoloogilise analüüsiga. Selleks teisendage järgneva teksti analüüs GT kujule ning printige välja iga tekstisõna koos selle mõlemas formaadis morfoloogilise analüüsiga (*form*).

*Kaks telgipäeva, 12. ja 13. juunil Tallinnas Vabaduse väljakul läksid väga edukalt. Saime kogeda nii vihma kui ka päikest, mürtsus puhkpillimuusika, ajakirjanikud sagisid ringi ning mis peamine – telgid olid tulvil heategudest. Kas ka sina käisid? Kui ei, tule järgmine aasta, paremat võimalust ei tule.*

Vastake küsimustele:
* Mis on suurim sisuline erinevus GT ja Filosofti analüüside vahel? Vihje: vaadake verbianalüüse
* Katseta *converter*'i loomisel lippe `disambiguate_sid_ksid=False` ja `disambiguate_neg=False`. Mis muutub?
* Kui lähtuda teksti morfoloogilise analüüsi täpsusest ja saagisest, siis millisel juhul oleks mõistlik kasutada GT kuju, millisel juhul Filosofti oma?

Analüüsi tõlgendamisel saab abi [siit](http://www.keeleveeb.ee/dict/corpus/shared/categories.html), soovi korral rohkem infot verbi morfoloogiliste kategooriate esitamise võimaluste kohta [siit](https://keeljakirjandus.ee/wp-content/uploads/2019/11/HJKaalep1_15.pdf).